In [1]:
import requests
import json
import datetime
import os

def kakao_login(code):
    url = "https://kauth.kakao.com/oauth/token"
    
    data = {
        "grant_type": "authorization_code",
        "client_id": "11d8ebef889239a83c2db142c8d841a6",
        "redirect_uri": "http://localhost:7777/kakaoLogin",
        "code": code
    }

    response = requests.post(url, data=data)

    if response.status_code != 200:
        print("에러!!!")
    else:
        tokens = response.json()
        print(tokens)
    
    return tokens
        
def save_tokens(filename, tokens):
    with open(filename, "w") as fp:
        json.dump(tokens, fp)
        
# 읽기
def load_tokens(filename):
    with open(filename) as fp:
        tokens = json.load(fp)
        
    return tokens

In [2]:
def getProfile(tokens):
    url = "https://kapi.kakao.com/v2/user/me"

    headers = {
        "Authorization": "Bearer " + tokens['access_token'],
    }

    response = requests.post(url, headers=headers)
    print(response.status_code)
    profile = response.json()

    if response.status_code != 200:
        print("에러!")
    else:
        print("카톡 메시지가 간드아!!!")
    
    return profile

In [ ]:
from flask import Flask, url_for, request, jsonify
from flask_cors import CORS, cross_origin

import json

app = Flask(__name__)
CORS(app)

@app.route('/kakao-login', methods = ['POST'])
def login():
    #Spring에서 code 값 받아오기
    params = json.loads(request.get_data())
    code = params['code']
    print('code:' + code)
    KAKAO_LOGIN_TOKEN_SAVED_FILE = "kakao_login/kakao_login_token.json"
    #토큰 획득
    authorize_tokens = kakao_login(code)
    save_tokens(KAKAO_LOGIN_TOKEN_SAVED_FILE, authorize_tokens)
    tokens = load_tokens(KAKAO_LOGIN_TOKEN_SAVED_FILE)
    
    
    #프로필 요청
    return getProfile(tokens)


if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


code:n3PeXeER0Db8ZEbBWGCy-UF3ZFkLHWP3NhLtifnQo46howwACJHln2qNfyEzhUti9Rn6VAopcBMAAAGBVVFIRQ


127.0.0.1 - - [12/Jun/2022 09:30:01] "POST /kakao-login HTTP/1.1" 200 -


{'access_token': '2PLt5K6Lz5A-4QGRA7QDDw8WlqZea23pElu6sqCXCj10aQAAAYFVUUq9', 'token_type': 'bearer', 'refresh_token': '5FqGdmIhoEJNyrypNLR4-jdamEHAElwi4Ve77pQ-Cj10aQAAAYFVUUq7', 'expires_in': 21599, 'scope': 'account_email profile_nickname', 'refresh_token_expires_in': 5183999}
200
카톡 메시지가 간드아!!!


...

code:N4PNSCiiX-ONyzjn5Kw0FJEfTUPjt9UBfugEJBlzm_kgmQ-5knUl4Pj7rjf_IwnMN8Q8kQopcBMAAAGBVVrFtg
{'access_token': 'qa0HWBxB6lOORCONVru3ZiavK7Poq0UI4soZZYhaCinI2AAAAYFVWsYF', 'token_type': 'bearer', 'refresh_token': 'iybU7diKPQ6d2b3_I_uumf8dDreKtS5wPVX4v0aCCinI2AAAAYFVWsYD', 'expires_in': 21599, 'scope': 'account_email profile_nickname', 'refresh_token_expires_in': 5183999}
200
카톡 메시지가 간드아!!!


127.0.0.1 - - [12/Jun/2022 09:40:46] "POST /kakao-login HTTP/1.1" 200 -


code:hdf3mtHFmaVF5yLjtfrgYkIN7rCnt22sGkhjLmDCeXz5PDPGjaqnI1ZmmJMSJjfdytAW0gopyNoAAAGBVVsgfw
{'access_token': 'sV7VkpNolzTGvnLEQeuqubob3FTJn1krRa2lA9yICj1z7AAAAYFVWyDZ', 'token_type': 'bearer', 'refresh_token': 'xSedyT2mOyD38VIGaYNdh7RNO3EPDo9Rk9hURwnECj1z7AAAAYFVWyDX', 'expires_in': 21599, 'scope': 'account_email profile_nickname', 'refresh_token_expires_in': 5183999}
200
카톡 메시지가 간드아!!!


127.0.0.1 - - [12/Jun/2022 09:41:26] "POST /kakao-login HTTP/1.1" 200 -


code:OgxpO2l_yCz2lsvfaCzSpFfIBohHXbzszls1j5vs4pcRNymYW33WvI3KnddYlmrJI_EZ1worDKcAAAGBVVu83A
{'access_token': 'wEch4UOfNRdBSJzs8zuMT5CnF9GAwopdL_gxDUAwCisNIAAAAYFVW70r', 'token_type': 'bearer', 'refresh_token': 'deU7U96OFc_r-0E6avGfeEst9PAkVRZ3l_dq4owmCisNIAAAAYFVW70q', 'expires_in': 21599, 'scope': 'account_email profile_nickname', 'refresh_token_expires_in': 5183999}
200
카톡 메시지가 간드아!!!
